In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter

In [ ]:
plt.rcParams["figure.figsize"] = (16, 3)
plt.rcParams["axes.grid"] = True

In [ ]:
def describe(frames, types, labels):
    """
    abbreaviated version of pd.describe in horizontal layout
    
    needs 3 *size-matched* lists
    
    :param frames: a list of frames to describe
    :param types: list of event types to describe
    :param labels: labels for values of each event types
    """
    storage = []
    
    for frame, type_n, label in zip(frames, types, labels):
        
        description = frame[frame.type == type_n].describe()
        description.rename(columns={'value': label}, inplace=True)
        storage.append(description)
        
    return pd.concat(storage, axis=1).loc[['min', 'max', 'mean', 'std', 'count']].transpose()

In [ ]:
def plot_timings(name_dict, title):

    pre_data = dt[dt['name'].isin(name_dict.keys())].copy()
    for k, v in name_dict.items():
        pre_data.loc[pre_data['name'] == k, 'name'] = v
    pre_data = pre_data.rename(columns={'name': 'process'})
    ax = sns.lineplot(data=pre_data, x='timestamp', y='elapsed', hue='process').set(title=title)
    plt.grid(False)
    plt.show()

In [ ]:
ds = pd.read_csv("../csv/session.csv", delimiter=';')
dg = pd.read_csv("../csv/global_returns.csv", delimiter=';')
dt = pd.read_csv("../csv/processing_timings.csv", delimiter=';')


# Session data

In [ ]:
print(f"ALS start time: {dg.iloc[0,0][:-4]}")
print(f"ALS stop time:  {dg.iloc[-1,0][:-4]}")


# Alignment

### translations

In [ ]:
ax = ds[ds.type == "x_trans"].plot(title="Image translations",
                                             x='timestamp', sharex=True,
                                             y='value',
                                             label="x",
                                             ylabel="translation (px)",
                                             figsize=(16, 2))

ds[ds.type == "y_trans"].plot(x='timestamp', y='value', label="y", ax=ax, xlabel="")

ax.xaxis.set_major_formatter(DateFormatter("%H:%M:%S"))

plt.show()

all_trans = [ds[(ds.type == "x_trans")], ds[(ds.type == "y_trans")]]

describe(all_trans, ['x_trans', 'y_trans'], ['translation x (px)', 'translation y (px)'])

In [ ]:
import math
rot=ds[ds.type == 'rotation'].copy()
rot['value'] = rot['value'].apply(lambda v:  math.degrees(v))


rot.plot(title="Image rotations",
    x='timestamp', y='value', legend=False, ylabel="Rotation (degrees)", xlabel='', figsize=(16, 2))
plt.show()

describe([rot], ['rotation'], ['rotation (°)'])

In [ ]:
scale = ds[ds.type == 'scale']
scale.plot(x="timestamp", y="value", legend=False,
                           figsize=(16, 2), title="Image scalings")
plt.show()

describe([scale], ['scale'], ['scale'])


In [ ]:
rat=ds[ds.type == 'ratio'].copy()
rat['value'] = rat['value'].apply(lambda v:  v * 100)

rat.plot(
    kind="bar", x='timestamp', y='value', label='ratio',
    ylabel="%",
    title="match detection surface ratio",
    xticks=[], xlabel="", legend=False, figsize=(16,1), ylim=(0, 100),
    grid=False
)
plt.show()

In [ ]:
matches = ds[ds.type == 'matches']
ax = matches.plot(
    x='timestamp',
    y='value',
    title="frame match count vs required minimum",
    ylim=(0,ds[ds.type == 'matches'].value.max() + 5),
    xlabel="",
    label="matches",
    figsize=(16, 2)
)

ds[ds.type == 'req_matches'].plot(ax=ax,x='timestamp', y='value', label='required', xlabel='')

plt.show()

describe([matches], ['matches'], ['matches'])

In [ ]:
ds[ds.type == 'align'].plot(
    kind="bar",
    grid=False,
    x='timestamp',
    ylabel="Accepted",
    xticks=[], xlabel="",
    legend=False,
    figsize=(16,1),
    title="frame compliance to min. required match count. 1: Accepted, 0: Rejected"
)
plt.show()

In [ ]:
ds[ds.type == 'align'].value.transform(lambda x: "accepted" if x == 1. else "rejected").value_counts().plot(
    kind="pie",
    autopct='%1.1f%%',
    title="accepted/rejected frames distribution",
    ylabel=""
)
plt.show()

# Queues

In [ ]:
def plot_queue_size(data, title):
    
    data.plot(kind='bar', title=title,
              x="timestamp",
              xlabel="",
              figsize=(16, 2),
              legend=False, xticks=[])
    plt.show()

In [ ]:
plot_queue_size(ds[ds.type == 'q_pre'], "Pre-processing queue size")

In [ ]:
plot_queue_size(ds[ds.type == 'q_stack'], "Stacking queue size")

In [ ]:
plot_queue_size(ds[ds.type == 'q_post'], "Post-processing queue size")

In [ ]:
plot_queue_size(ds[ds.type == 'q_save'], "Save queue size")

In [ ]:
data = ds[ds.type.str.startswith('q_')].copy()
data.replace(to_replace={
    "q_pre": "Pre-processing",
    "q_stack": "Stacking",
    "q_post": "Post-processing",
    "q_save": "Save",
    }, inplace=True)

print(f'{data.shape[0]} queue events logged')
plt.figure(figsize=(16, 6))
plt.grid(False)
ax = sns.barplot(data = data,
                  x="timestamp",
                  y='value',
                  hue='type')
ax.set_title("Combined queue sizes")
plt.show()

# Timings

In [ ]:
total = ds[ds.type == 'frm_total']
total.plot(title="Total frame processin time",
                                x="timestamp",
                                ylabel="Processing time (s)", figsize=(16, 6), legend=False)
plt.show()
describe([total], ['frm_total'], ['Total frame processin time'])

In [ ]:
prepro_name_dict = {
    'HotPixelRemover.process_image()': 'H.P. Remove',
    'RemoveDark.process_image()': 'Dark remove',
    'Debayer.process_image()': 'Debayer',
    'Standardize.process_image()': 'Nomalize',
    'read_disk_image()': 'Load'
}

plot_timings(prepro_name_dict, "Pre-processing time")

In [ ]:
stack_name_dict = {
    "Stacker._find_transformation()": "Find transf.",
    "Stacker._apply_transformation()": "Apply Transf.",
    "Stacker._stack_image()": "Stack"
}

plot_timings(stack_name_dict, "Stacking time")

In [ ]:
old_new_pp_names_dict = {
    'AutoStretch.process_image()': 'Stretch',
    'Levels.process_image()': 'Levels',
    'ColorBalance.process_image()': 'Col. Balance',
    'ImageSaver._save_image()': 'Save'
}

plot_timings(old_new_pp_names_dict, "Post-process timing")